# imports

In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

В этом домашнем задании вы будете решать задачу классификации оттоковых клиентов телекома: для каждого клиента необходимо определить, перестанет он пользоваться услугами оператора в ближайшее время или нет.

In [12]:
df = pd.read_csv("https://raw.githubusercontent.com/evgpat/edu_stepik_first_step_in_nlp/main/datasets/telecom_churn.csv")
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


Сделаем первичную обработку данных:

1) Выбросите колонку State

**Вопрос:** сколько столбцов осталось в матрице df?

In [13]:
df.drop('State', axis=1, inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Account length          3333 non-null   int64  
 1   Area code               3333 non-null   int64  
 2   International plan      3333 non-null   object 
 3   Voice mail plan         3333 non-null   object 
 4   Number vmail messages   3333 non-null   int64  
 5   Total day minutes       3333 non-null   float64
 6   Total day calls         3333 non-null   int64  
 7   Total day charge        3333 non-null   float64
 8   Total eve minutes       3333 non-null   float64
 9   Total eve calls         3333 non-null   int64  
 10  Total eve charge        3333 non-null   float64
 11  Total night minutes     3333 non-null   float64
 12  Total night calls       3333 non-null   int64  
 13  Total night charge      3333 non-null   float64
 14  Total intl minutes      3333 non-null   

Сделаем простую кодировку признаков: 

2) Колонку International plan закодируем следующим образом: "Yes" замените на 1, а "No" на 0.

In [15]:
df['International plan'] = df['International plan'].map({'Yes' : 1, 'No' : 0})

Аналогичным образом закодируйте столбец Voice mail plan.

In [16]:
df['Voice mail plan'] = df['Voice mail plan'].map({'Yes' : 1, 'No' : 0})

3) Переведем целевую переменную (Churn) в числа: True в 1, а False в 0.

In [17]:
df['Churn'] = df['Churn'].astype(int)

Выведите на экран первые строки измененной таблицы, чтобы проверить, что все преобразования сработали.

In [18]:
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


Создайте матрицу объект-признак X и вектор с целевой переменной (Churn) y.

In [19]:
X = df.drop('Churn', axis=1)
y = df['Churn']

Проверьте баланс классов. Является ли выборка сбалансированной?

**Вопрос:** сколько объектов относятся к классу 1?

In [41]:
y.value_counts()

0    2850
1     483
Name: Churn, dtype: int64

Разбейте данные на тренировочную и тестовую части, размер тестовой части - 30%.

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=True)

Для линейных моделей очень важно масштабирование признаков. Поэтому 
отмасштабируйте признаки при помощи StandardScaler, обучив метод на тренировочных данных и применив его и к трейну, и к тесту.

Для восстановления DataFrame-ов из преобразованных объектов используйте код с занятия.

In [43]:
ss = StandardScaler()

In [45]:
ss.fit(X_train)

X_train_transformed = ss.transform(X_train)
X_test_transformed = ss.transform(X_test)

In [46]:
X_train_transformed

array([[-0.03706973, -0.69064032, -0.31860769, ..., -0.60951022,
        -0.98136561, -0.42633947],
       [ 0.58637088, -0.52555423, -0.31860769, ...,  0.21794823,
         0.59488172, -1.1827273 ],
       [-0.26150835, -0.69064032, -0.31860769, ...,  0.63167746,
         0.78189412,  0.33004835],
       ...,
       [-0.18669548, -0.52555423, -0.31860769, ..., -0.195781  ,
         1.42307947, -0.42633947],
       [ 0.96043524,  1.71489992, -0.31860769, ..., -1.02323945,
        -0.01958758,  2.59921184],
       [ 0.78587187, -0.52555423, -0.31860769, ...,  1.87286514,
         0.02048651, -0.42633947]])

In [47]:
X_train = pd.DataFrame(ss.transform(X_train), columns=X.columns)
X_test = pd.DataFrame(ss.transform(X_test), columns=X.columns)

X_train.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
0,-0.037070,-0.690640,-0.318608,-0.601613,-0.576508,-2.316853,-0.419491,-2.316497,-0.550812,0.158921,-0.549703,-0.117562,1.277071,-0.117372,-0.987351,-0.609510,-0.981366,-0.426339
1,0.586371,-0.525554,-0.318608,-0.601613,-0.576508,0.248079,0.122765,0.248018,0.797779,0.815228,0.798884,0.520995,2.687370,0.522045,0.599572,0.217948,0.594882,-1.182727
2,-0.261508,-0.690640,-0.318608,-0.601613,-0.576508,0.764014,-0.370195,0.763956,-0.273161,0.158921,-0.272053,1.601023,1.226704,1.599417,0.779904,0.631677,0.781894,0.330048
3,-1.508390,1.714900,-0.318608,-0.601613,-0.576508,-1.821188,1.108684,-1.821153,0.186947,0.411347,0.187587,0.215512,-1.795365,0.215475,0.383173,-0.609510,0.381153,3.355600
4,-0.136820,-0.525554,-0.318608,1.662198,2.281281,1.478952,1.453756,1.479332,0.250410,0.562802,0.250583,-0.107708,-0.133228,-0.108612,-0.049624,0.217948,-0.046304,-0.426339


Обучите на тренировочных данных с параметрами по умолчанию:

1) Логистическую регрессию

2) Метод ближайших соседей

In [50]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [51]:
pred_test = model.predict(X_test)

In [53]:
pred_test[:5]

array([0, 0, 0, 0, 0])

In [55]:
accuracy_score(y_test, pred_test)

0.847

In [58]:
modelKNN = KNeighborsClassifier()
modelKNN.fit(X_train, y_train)

pred_test_KNN = modelKNN.predict(X_test)
accuracy_score(y_test, pred_test_KNN)

0.884

С помощью обученных моделей сделайте предсказание на тестовых данных. Вычислите значение метрики roc-auc на тесте. 

Не забудьте, что для корректного вычисления roc-auc необходимо предсказывать не сами классы, а их вероятности.

**Вопрос:** чему равен ROC-AUC логистической регрессии, округленный до десятых?

In [60]:
proba = model.predict_proba(X_test)
proba[:5]

array([[0.82478161, 0.17521839],
       [0.95346031, 0.04653969],
       [0.89140612, 0.10859388],
       [0.96434522, 0.03565478],
       [0.90895294, 0.09104706]])

In [61]:
probaKNN = modelKNN.predict_proba(X_test)
probaKNN[:5]

array([[1. , 0. ],
       [0.4, 0.6],
       [1. , 0. ],
       [0.8, 0.2],
       [1. , 0. ]])

In [62]:
score = roc_auc_score(y_test, proba[:,1])
score

0.7888283928211333

In [63]:
score = roc_auc_score(y_test, probaKNN[:,1])
score

0.8275458761847146

## Бонус. Подбор гиперпараметров модели.

У моделей машинного обучения есть гиперпараметры - это величины, которые задает человек до начала обучения моделей. Подбор гиперпараметров может значительно влиять на качество моделей.

Про эту тему можно посмотреть [на Stepik](https://stepik.org/lesson/806516/step/4?unit=809692).

Теперь ваша цель - максимально улучшить модели. Для этого:

1) у логистической регрессии подберите гиперпараметр С (переберите 20 значений C в диапазоне от 0.1 до 10 с равным шагом).

2) у метода ближайших соседей подберите число соседей n_neighbors (ищите значение гиперпараметра в диапазоне от 1 до 20 с шагом 1).

Подбор гиперпараметров осуществляется только по тренировочным данным!

После подбора гиперпараметров обучите модели с найденными гиперпараметрами на тренировочных данных, сделайте предсказания на тесте и оцените их качество (roc_auc).

In [ ]:
# ваш код здесь

Какая модель справилась с решением задачи лучше всего по метрике roc_auc?

Нарисуйте roc-кривую для наилучшей модели.

In [ ]:
# ваш код здесь

Сравните между собой две лучшие модели:

1) насколько сильно отличается качество предсказания на тестовых данных?
(вычислите разность значений roc-auc на тестовых данных)

2) постройте гистограмму важности признаков для логистической регрессии. Какие три признака наиболее важны для модели?

In [ ]:
# ваш код здесь

Так можно строить гистограмму

In [ ]:
features = X.columns
importances = np.abs(lr.coef_[0])
indices = np.argsort(importances)

plt.title('Feature Importances of LogisticRegression')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

Сделайте выводы:

1) Являются ли данные сбалансированными? Что это означает?

2) Какая модель после подбора гиперпараметров лучше всего решает задачу с точки зрения метрики roc-auc?